# Building Agents with Ollama: A Step-by-Step Guide

In [12]:
%pip install termcolor

Note: you may need to restart the kernel to use updated packages.


In [13]:
from termcolor import colored
import operator
import json
import requests
import os

In [14]:
agent_system_prompt_template = """
You are an agent with access to a toolbox. Given a user query, 
you will determine which tool, if any, is best suited to answer the query. 
You will generate the following JSON response:

"tool_choice": "name_of_the_tool",
"tool_input": "inputs_to_the_tool"

- `tool_choice`: The name of the tool you want to use. It must be a tool from your toolbox 
                or "no tool" if you do not need to use a tool.
- `tool_input`: The specific inputs required for the selected tool. 
                If no tool, just provide a response to the query.

Here is a list of your tools along with their descriptions:
{tool_descriptions}

Please make a decision based on the provided user query and the available tools.
"""

In [15]:
def setup_ollama_model(model, system_prompt, temperature=0, stop=None):
    """
    Sets up the Ollama model configuration.

    Parameters:
    model (str): The name of the model to use.
    system_prompt (str): The system prompt to use.
    temperature (float): The temperature setting for the model.
    stop (str): The stop token for the model.

    Returns:
    dict: Configuration for the Ollama model.
    """
    return {
        "model_endpoint": "http://localhost:11434/api/generate",
        "model": model,
        "system_prompt": system_prompt,
        "temperature": temperature,
        "headers": {"Content-Type": "application/json"},
        "stop": stop,
    }


# Example configuration
ollama_config = setup_ollama_model(
    model="mistral", system_prompt=agent_system_prompt_template
)

In [16]:
def basic_calculator(input_str):
    """
    Perform a numeric operation on two numbers based on the input string.

    Parameters:
    input_str (str): A JSON string representing a dictionary with keys 'num1', 'num2', and 'operation'.
                     Example: '{"num1": 5, "num2": 3, "operation": "add"}' or "{'num1': 67869, 'num2': 9030393, 'operation': 'divide'}"

    Returns:
    str: The formatted result of the operation.

    Raises:
    Exception: If an error occurs during the operation (e.g., division by zero).
    ValueError: If an unsupported operation is requested or input is invalid.
    """
    # Clean and parse the input string
    try:
        # Replace single quotes with double quotes
        input_str_clean = input_str.replace("'", '"')
        # Remove any extraneous characters such as trailing quotes
        input_str_clean = input_str_clean.strip().strip('"')

        input_dict = json.loads(input_str_clean)
        num1 = input_dict["num1"]
        num2 = input_dict["num2"]
        operation = input_dict["operation"]
    except (json.JSONDecodeError, KeyError) as e:
        return str(e), "Invalid input format. Please provide a valid JSON string."

    # Define the supported operations
    operations = {
        "add": operator.add,
        "subtract": operator.sub,
        "multiply": operator.mul,
        "divide": operator.truediv,
        "floor_divide": operator.floordiv,
        "modulus": operator.mod,
        "power": operator.pow,
        "lt": operator.lt,
        "le": operator.le,
        "eq": operator.eq,
        "ne": operator.ne,
        "ge": operator.ge,
        "gt": operator.gt,
    }

    # Check if the operation is supported
    if operation in operations:
        try:
            # Perform the operation
            result = operations[operation](num1, num2)
            result_formatted = (
                f"\n\nThe answer is: {result}.\nCalculated with basic_calculator."
            )
            return result_formatted
        except Exception as e:
            return str(e), "\n\nError during operation execution."
    else:
        return "\n\nUnsupported operation. Please provide a valid operation."

In [17]:
def reverse_string(input_string):
    """
    Reverse the given string.

    Parameters:
    input_string (str): The string to be reversed.

    Returns:
    str: The reversed string.
    """
    # Reverse the string using slicing
    reversed_string = input_string[::-1]

    reversed_string = f"The reversed string is: {reversed_string}\n\n.Executed using the reverse_string function."
    # print (f"DEBUG: reversed_string: {reversed_string}")
    return reversed_string

In [18]:
def prepare_tools(tools):
    """
    Stores the tools and returns their descriptions.

    Parameters:
    tools (list): List of tool functions.

    Returns:
    str: Description of the tools.
    """
    tools_dict = {tool.__name__: tool.__doc__ for tool in tools}
    tools_description = "\n".join(
        f'{name}: "{desc}"' for name, desc in tools_dict.items()
    )
    return tools_description


# Define tools
tools = [basic_calculator, reverse_string]
tools_description = prepare_tools(tools)

In [19]:
def generate_response(prompt, config):
    """
    Generates a response from the Ollama model based on the provided prompt.

    Parameters:
    prompt (str): The user query.
    config (dict): The Ollama model configuration.

    Returns:
    dict: The response from the model.
    """
    agent_system_prompt = agent_system_prompt_template.format(
        tool_descriptions=tools_description
    )
    payload = {
        "model": config["model"],
        "format": "json",
        "prompt": prompt,
        "system": agent_system_prompt,
        "stream": False,
        "temperature": config["temperature"],
        "stop": config["stop"],
    }

    try:
        response = requests.post(
            config["model_endpoint"],
            headers=config["headers"],
            data=json.dumps(payload),
        )
        response_json = response.json()
        response_dict = json.loads(response_json["response"])

        print(f"\n\nResponse from Ollama model: {response_dict}")
        return response_dict
    except requests.RequestException as e:
        print(f"Error in invoking model! {str(e)}")
        return {"error": str(e)}

In [20]:
def execute_tool(response, tools):
    """
    Executes the appropriate tool based on the model's response.

    Parameters:
    response (dict): The model's response.
    tools (list): List of available tool functions.

    Returns:
    None
    """
    tool_choice = response.get("tool_choice")
    tool_input = response.get("tool_input")

    for tool in tools:
        if tool.__name__ == tool_choice:
            result = tool(tool_input)
            print(colored(result, "cyan"))
            return

    print(colored(tool_input, "cyan"))

In [21]:
prompt = "What is 5+5?"
response = generate_response(prompt, ollama_config)
execute_tool(response, tools)



Response from Ollama model: {'tool_choice': 'basic_calculator', 'tool_input': '{"num1": 5, "num2": 5, "operation": "add"}'}


The answer is: 10.
Calculated with basic_calculator.
